In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/ner-dataset/NER_Dataset.csv')

In [2]:
df

,Sentence_ID,Word,POS,Tag
0,Sentence: 1,"['Thousands', 'of', 'demonstrators', 'have', '...","['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 10,"['Iranian', 'officials', 'say', 'they', 'expec...","['JJ', 'NNS', 'VBP', 'PRP', 'VBP', 'TO', 'VB',...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '..."
2,Sentence: 100,"['Helicopter', 'gunships', 'Saturday', 'pounde...","['NN', 'NNS', 'NNP', 'VBD', 'JJ', 'NNS', 'IN',...","['O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', '..."
3,Sentence: 1000,"['They', 'left', 'after', 'a', 'tense', 'hour-...","['PRP', 'VBD', 'IN', 'DT', 'NN', 'JJ', 'NN', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 10000,"['U.N.', 'relief', 'coordinator', 'Jan', 'Egel...","['NNP', 'NN', 'NN', 'NNP', 'NNP', 'VBD', 'NNP'...","['B-geo', 'O', 'O', 'B-per', 'I-per', 'O', 'B-..."
...,...,...,...,...
47954,Sentence: 9995,"['Opposition', 'leader', 'Mir', 'Hossein', 'Mo...","['NNP', 'NN', 'NNP', 'NNP', 'NNP', 'VBZ', 'VBN...","['O', 'O', 'O', 'B-per', 'I-per', 'O', 'O', 'O..."
47955,Sentence: 9996,"['On', 'Thursday', ',', 'Iranian', 'state', 'm...","['IN', 'NNP', ',', 'JJ', 'NN', 'NNS', 'VBN', '...","['O', 'B-tim', 'O', 'B-gpe', 'O', 'O', 'O', 'O..."
47956,Sentence: 9997,"['Following', 'Iran', ""'s"", 'disputed', 'June'...","['VBG', 'NNP', 'POS', 'JJ', 'NNP', 'CD', 'NNS'...","['O', 'B-geo', 'O', 'O', 'B-tim', 'I-tim', 'O'..."
47957,Sentence: 9998,"['Since', 'then', ',', 'authorities', 'have', ...","['IN', 'RB', ',', 'NNS', 'VBP', 'VBN', 'JJ', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


# EDA
Investigate how many Tags are in the dataset

In [3]:
import ast 

# Current data has list form read in literal. Change this to list form
for i, row in df.iterrows():
    df.loc[i, 'Word'] = ast.literal_eval(df.loc[i, 'Word'])
    df.loc[i, 'Tag'] = ast.literal_eval(df.loc[i, 'Tag'])

unique_tags = set()

def add_unique_tags(row):
    taglist = row['Tag']
    for tag in taglist:
        # Add the tag only if not in the unique_tags set
        if tag not in unique_tags:
            unique_tags.add(tag)

df.apply(add_unique_tags, axis=1)
unique_tags

{'B-art',
 'B-eve',
 'B-geo',
 'B-gpe',
 'B-nat',
 'B-org',
 'B-per',
 'B-tim',
 'I-art',
 'I-eve',
 'I-geo',
 'I-gpe',
 'I-nat',
 'I-org',
 'I-per',
 'I-tim',
 'O'}

In [4]:
# Let's hardcode the tags to reorder
tags = ['O', 'B-art', 'I-art', 'B-eve', 'I-eve', 'B-geo', 'I-geo', 'B-gpe', 'I-gpe', 
        'B-nat', 'I-nat', 'B-org', 'I-org', 'B-per', 'I-per', 'B-tim', 'I-tim']

In [5]:
import sklearn
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Word'], df['Tag'], test_size=0.2, random_state=42)

In [6]:
X_train[0]

['Thousands',
 'of',
 'demonstrators',
 'have',
 'marched',
 'through',
 'London',
 'to',
 'protest',
 'the',
 'war',
 'in',
 'Iraq',
 'and',
 'demand',
 'the',
 'withdrawal',
 'of',
 'British',
 'troops',
 'from',
 'that',
 'country',
 '.']

# Make Vocabulary
(Note: 'O' tagged tokens will be excluded from the vocabulary)

In [7]:
def make_vocab(x_data, y_labels):
    '''
    Args:
        :x_data: (train_data_length, sentence_length)
        :y)data: (train_data_length, 1)
    '''
    vocab = set()
    
    # Loop through each sentence
    for sentence, tags in zip(x_data, y_labels):
        # Loop through each token
        for token, tag in zip(sentence, tags):
            if tag != 'O':
                vocab.add(token)
                
    vocab = list(vocab)
    return vocab

In [8]:
vocab_train = make_vocab(X_train, y_train)
vocab_test = make_vocab(X_test, y_test)

In [9]:
print(f"Length of vocabulary of train dataset: {len(vocab_train)}")
print(f"Length of vocabulary of test dataset:  {len(vocab_test)}")

Length of vocabulary of train dataset: 12061
Length of vocabulary of test dataset:  5686


In [10]:
# Checking how many tokens from test vocab is NOT in train vocab
count = 0
for i in range(len(vocab_test)):
    if vocab_test[i] not in vocab_train:
        count += 1

print(f"{count} Tokens from vocab_test are not in vocab_train")

1498 Tokens from vocab_test are not in vocab_train


# Encoding
Encoding each token to integers
* Special Tokens x->y mapping:
    * '\<PAD\>' -> '\<PAD\>'
    * '\<UNK\>' -> 'O'

In [11]:
def word2idx(special_tokens, vocab):
    num_spec_tokens = len(special_tokens)
    # Word2idx
    word_to_idx = {spec: idx for idx, spec in enumerate(special_tokens)} # Adding special tokens
    word_to_idx.update({token: (idx + num_spec_tokens) for idx, token in enumerate(vocab)}) # Adding non-special vocab tokens
    # idx2Word
    idx_to_word = {idx: token for token, idx in word_to_idx.items()} # Key-value swapping
    
    return word_to_idx, idx_to_word

In [12]:
token2idx, idx2token = word2idx(['<PAD>', '<UNK>'], vocab_train)
tag2idx, idx2tag = word2idx(['<PAD>'], tags)

In [13]:
tag2idx

{'<PAD>': 0,
 'O': 1,
 'B-art': 2,
 'I-art': 3,
 'B-eve': 4,
 'I-eve': 5,
 'B-geo': 6,
 'I-geo': 7,
 'B-gpe': 8,
 'I-gpe': 9,
 'B-nat': 10,
 'I-nat': 11,
 'B-org': 12,
 'I-org': 13,
 'B-per': 14,
 'I-per': 15,
 'B-tim': 16,
 'I-tim': 17}

In [14]:
# Encoding Tokenized data
def encode_tokens(tokenized_data, token2idx, unknown_token='<UNK>'):
    '''
    Args:
        :tokenized_data: (dataset_length, sequence_length)
        :token2idx: {token : idx}
        :unknown_token: The token that the unknown tokens(not in vocab e.g. token2idx) will be mapped to 
    Returns:
        :encoded_data: listof(listof(int))
    '''
    encoded_data = []
    
    for sent in tokenized_data:
        encoded_sent = []
        for token in sent:
            try:
                encoded_sent.append(token2idx[token])
            except KeyError: # If not found, it means unkown token
                encoded_sent.append(token2idx[unknown_token])
        # Adding encoded sentence to the final output
        encoded_data.append(encoded_sent)
    
    return encoded_data

In [15]:
# X dataset
enc_X_train = encode_tokens(X_train, token2idx, '<UNK>')
enc_X_test = encode_tokens(X_test, token2idx, '<UNK>')
# y dataset
enc_y_train = encode_tokens(y_train, tag2idx, 'O')
enc_y_test = encode_tokens(y_test, tag2idx, 'O')

In [16]:
enc_X_train

[[8546,
  798,
  2740,
  1,
  6535,
  1,
  1,
  5806,
  1,
  6438,
  5698,
  1,
  74,
  11448,
  4641,
  1,
  5636,
  1,
  1,
  6808,
  1,
  1],
 [1,
  7272,
  8284,
  1,
  134,
  1,
  1,
  1,
  1,
  4641,
  1,
  1,
  1,
  7272,
  1,
  2873,
  1,
  1,
  4641,
  2873,
  3628,
  1,
  7272,
  1,
  1],
 [1697,
  1,
  908,
  1,
  1,
  1,
  1,
  1,
  7408,
  7272,
  736,
  356,
  1,
  10447,
  1,
  4641,
  1,
  2873,
  1,
  1,
  1,
  1,
  1],
 [5970, 2740, 10608, 6756, 1, 7272, 1, 8365, 1, 10353, 1],
 [1,
  1,
  5636,
  1,
  1,
  1,
  1,
  4641,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  504,
  1,
  159,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2740,
  10270,
  1,
  1,
  6493,
  1,
  6262,
  1,
  10270,
  1,
  1],
 [10132,
  10039,
  8333,
  9391,
  1,
  1,
  5636,
  1,
  1,
  945,
  1,
  1,
  5636,
  1,
  10298,
  1,
  1,
  1,
  10298,
  5636,
  1,
  7272,
  5296,
  1197,
  10447,
  7171,
  1],
 [1697,
  11768,
  9181,
  2873,
  7959,
  1,
  1,
  863,
  1,
  908,
  1,
  159,
  7272,
  

# Determine Context Length

In [17]:
import matplotlib.pyplot as plt

def set_sent_pad(encode_x, context_length, report=False):
    if report:
        print(f'훈련 데이터셋 최대 길이: {max(len(exam) for exam in encode_x)}')
        print(f'훈련 데이터셋 평균 길이: {sum(map(len, encode_x)) / len(encode_x):.2f}')

        plt.hist([len(exam) for exam in encode_x], bins=50)
        plt.xlabel('length of samples')
        plt.ylabel('number of samples')
        plt.show()

    def below_th_len(encode_x, context_length):
        cnt = 0
        for sent in encode_x:
            if(len(sent) <= context_length):
                cnt +=1
        
        print(f'데이터셋 문장 길이가 {context_length} 이하 데이터 비율: '+
              f'{cnt / len(encode_x)*100:.2f}%')
        
    below_th_len(encode_x, context_length)
    
set_sent_pad(enc_X_train, 40)

데이터셋 문장 길이가 40 이하 데이터 비율: 98.40%


# Padding

In [18]:
def pad_seq(dataset, context_length):
    padded = np.zeros((len(dataset), context_length), dtype=int)
    for idx, sent in enumerate(dataset):
        if len(sent) != 0:
            padded[idx, :len(sent)] = np.array(sent)[:context_length]
    return padded

In [19]:
CONTEXT_LENGTH = 40
# X datasets
pad_X_train = pad_seq(enc_X_train, CONTEXT_LENGTH)
pad_X_test = pad_seq(enc_X_test, CONTEXT_LENGTH)
# y datasets
pad_y_train = pad_seq(enc_y_train, CONTEXT_LENGTH)
pad_y_test = pad_seq(enc_y_test, CONTEXT_LENGTH)

In [20]:
print(f"{pad_X_train.shape}")
print(f"{pad_y_train.shape}")

(38367, 40)
(38367, 40)


# DataLoader

In [21]:
import torch
from torch.utils.data import TensorDataset, DataLoader

def set_dataloader(x_data, y_label, bs, training=False):
    tensor_x = torch.tensor(x_data, dtype=torch.int64)
    tensor_y = torch.tensor(y_label, dtype=torch.int64)
    
    dataset = TensorDataset(tensor_x, tensor_y)
    
    shuffle=False
    if training:
        shuffle=True
    
    dataloader = DataLoader(dataset, shuffle=shuffle, batch_size=bs)
    
    return dataloader

In [22]:
BATCH_SIZE = 128

train_dataloader = set_dataloader(pad_X_train, pad_y_train, bs=BATCH_SIZE, training=True)
test_dataloader = set_dataloader(pad_X_test, pad_y_test, bs=BATCH_SIZE, training=False)